# Python interface

For this tutorial, we will use the [test data](https://github.com/JaGeo/LobsterPy/tests/test_data) that can be downloaded from our git repository using the following commands.

```bash
git clone --filter=blob:none --no-checkout https://github.com/JaGeo/LobsterPy.git
cd LobsterPy
git sparse-checkout set tests/test_data
git read-tree -mu HEAD
```

## Usage of Analysis, Description class and automatic plotting

### Basic usage : Analysis, Description

Lets first import the necessary modules

In [ ]:
from pathlib import Path
from lobsterpy.cohp.analyze import Analysis
from lobsterpy.cohp.describe import Description
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Directory of your VASP and Lobster computations
directory = Path(".") / ".." / ".." / "tests" / "test_data" / "CdF_comp_range"

```python
#### Change directory to your Lobster computations (Change this cell block type to Code and remove formatting when executing locally)
directory = Path("LobsterPy") / "tests" / "test_data" / "CdF_comp_range"
```

In [ ]:
# Initialize Analysis object
analyse = Analysis(
    path_to_poscar=directory / "POSCAR.gz",
    path_to_icohplist=directory / "ICOHPLIST.lobster.gz",
    path_to_cohpcar=directory / "COHPCAR.lobster.gz",
    path_to_charge=directory / "CHARGE.lobster.gz",
    which_bonds="cation-anion",
)

In [ ]:
# Initialize Description object and to get text description of the analysis
describe = Description(analysis_object=analyse)
describe.write_description()

In [ ]:
# Get static plots for detected relevant bonds
describe.plot_cohps(ylim=[-10, 2], xlim=[-4, 4])

In [ ]:
# Get interactive plots of relevant bonds, 

# Setting label_resolved arg to True will plot each COHP curve separately, alongside summed COHP for the bonds.
fig = describe.plot_interactive_cohps(label_resolved=True, hide=True)
fig.show(renderer='notebook')

Due to sphinx rendering limitations for Plotly figures, the output is not directly visible within the notebook; please use the link to see the [interactive label resolved plot](tutorial_assets/CdF2.html)

In [ ]:
# Dict summarizing the automatic analysis results
analyse.condensed_bonding_analysis

In [ ]:
# Dict with bonds identified
analyse.final_dict_bonds

In [ ]:
# Dict with ions and their co-ordination environments
analyse.final_dict_ions

:::{note}
You can also perform automatic analysis using `COBICAR(ICOBILIST.lobster)` or `COOPCAR(ICOOPLIST.lobster)`. You would need to set `are_cobis`/`are_coops` to `True` depending on the type of file you decide to analyze when you initialize the Analysis object. Change the default `noise_cutoff` value to 0.001 or lower, as ICOOP and ICOBI typically have smaller values and different units than ICOHP. Below is an example code snippet.
:::

```python
analyse = Analysis(
    path_to_poscar=directory / "POSCAR.gz",
    path_to_icohplist=directory / "ICOBILIST.lobster.gz",
    path_to_cohpcar=directory / "COBICAR.lobster.gz",
    path_to_charge=directory / "CHARGE.lobster.gz",
    which_bonds="cation-anion",
    are_cobis=True,
    noise_cutoff=0.001,
)
```

Accessing other results is the same as the above.

### Advanced usage : Analysis, Description

LobsterPy now also allows for automatic orbital-wise analysis and plotting of COHPs, COBIs, and COOPs. To switch on orbital-wise analysis, one must set `orbital_resolved` arg to `True`. By default, orbitals contributing 5% or more relative to summed ICOHPs are considered in the analysis. One can change this default threshold using the `orbital_cutoff` argument. Here, we will set this cutoff value to 3%.

In [ ]:
analyse = Analysis(
    path_to_poscar=directory / "POSCAR.gz",
    path_to_icohplist=directory / "ICOHPLIST.lobster.gz",
    path_to_cohpcar=directory / "COHPCAR.lobster.gz",
    path_to_charge=directory / "CHARGE.lobster.gz",
    which_bonds="cation-anion",
    orbital_resolved=True,
    orbital_cutoff=0.03,
)

In [ ]:
# Access the dict summarizing the results including orbital-wise analysis data 
analyse.condensed_bonding_analysis

In the above output, you will now see a key named `orbital_data` associated with each relevant bond identified. The `orbital_summary_stats` key contains the orbitals that contribute the most to the bonding and antibonding interactions, and values are reported there in percent.

:::{note}
You can get plots from orbital resolved analysis only when `orbital_resolved` arg to `True` when initializing the Analysis object. If this is not done, you will run into errors. Also, only the interactive plotter will plot the results of orbital resolved analysis, as static plots will not be much readable. In any case, you can generate static plots if you need to. You will learn how to use the plotters available in LobsterPy further in the Plotter usage section of the tutorial.
:::

In [ ]:
# Initialize the Description object
describe = Description(analysis_object=analyse)
describe.write_description()

In [ ]:
# Automatic interactive plots
fig = describe.plot_interactive_cohps(orbital_resolved=True, ylim=[-15,5], hide=True)
fig.show(renderer='notebook')

Due to sphinx rendering limitations for Plotly figures, the output is not directly visible within the notebook; please use the link to see the [interactive orbital resolved plot](tutorial_assets/CdF2_orb.html)

### Get LOBSTER calculation quality and description

This utility provides a quick overview of your LOBSTER calculation quality by reading the charge spilling and band overlaps file (if these are generated during LOBSTER runs). Optionally, one can obtain atom charge classification comparisons with the BVA method and a comparison between DOS from LOBSTER and VASP.

:::{note}
The DOS comparisons and basis set utilized analysis are now limited to VASP calculations only. Support for other code output will be added in the future.
:::

In [ ]:
# Directory to your VASP and Lobster computations
directory = Path(".") / ".." / ".." / "tests" / "test_data" / "K3Sb"

```python
#### Change directory to your Lobster computations (Change this cell block type to Code and remove formatting when executing locally)
directory = Path("LobsterPy") / "tests" / "test_data" / "K3Sb"
```

In [ ]:
# Get calculation quality summary dict
calc_quality_K3Sb = Analysis.get_lobster_calc_quality_summary(
            path_to_poscar=directory / "POSCAR.gz",
            path_to_charge=directory / "CHARGE.lobster.gz",
            path_to_lobsterin=directory / "lobsterin.gz",
            path_to_lobsterout=directory / "lobsterout.gz",
            potcar_symbols=["K_sv", "Sb"], # if POTCAR exists, then provide path_to_potcar and set this to None 
            path_to_bandoverlaps=directory / "bandOverlaps.lobster.gz",
            dos_comparison=True, # set to false to disable DOS comparisons 
            bva_comp=True, # set to false to disable LOBSTER charge classification comparisons with BVA method
            path_to_doscar=directory / "DOSCAR.LSO.lobster.gz",
            e_range=[-20, 0],
            path_to_vasprun=directory / "vasprun.xml.gz",
            n_bins=256,
        )
calc_quality_K3Sb

In [ ]:
# Get a text description from calculation quality summary dictionary
calc_quality_k3sb_des = Description.get_calc_quality_description(
            calc_quality_K3Sb
        )
Description.write_calc_quality_description(calc_quality_k3sb_des)

## Using plotting utilities

In [ ]:
from matplotlib import style
from pymatgen.io.lobster import Doscar
from lobsterpy.plotting import InteractiveCohpPlotter, IcohpDistancePlotter, PlainCohpPlotter, PlainDosPlotter, get_style_list

You can alter the appearance of the static plots using the style sheet that comes with LobsterPy or use any of the readily available matplotlib style sheets.

### Plot COHPs / COBIS / COOPs from Analysis object

The `are_cobis`/`are_coops` arg must be set to `True` in the plotter depending on the type of files you analyze or want to plot. Here, we will keep them false as we are plotting COHPs.

In [ ]:
# Using PlainCohpPlotter to get static plots of relevant bonds from Analysis object

style.use(get_style_list()[0]) # Use the LobsterPy style sheet for the generated plots

cohp_plot_static = PlainCohpPlotter(are_cobis=False, are_coops=False)
for plot_label, label_list in analyse.get_site_bond_resolved_labels().items():
    cohp = analyse.chemenv.completecohp.get_summed_cohp_by_label_list(label_list=label_list)
    cohp_plot_static.add_cohp(plot_label, cohp)
cohp_plot_static.get_plot(ylim=[-15,2]);

:::{note}
You can get plots from orbital resolved analysis only when `orbital_resolved` arg is set to `True` when initializing the Analysis object.
:::

In [ ]:
# Using PlainCohpPlotter to get static plots of relevant orbitals COHPs from Analysis object

style.use('default') # Complete reset the matplotlib figure style
style.use('seaborn-v0_8-ticks') # use one of the existing matplotlib style sheet

cohp_plot_static = PlainCohpPlotter()
for plot_label , orb_data in analyse.get_site_orbital_resolved_labels().items():
    for orb, label_list in orb_data.items():
        cohp = analyse.chemenv.completecohp.get_summed_cohp_by_label_and_orbital_list(label_list=label_list, 
                                                                                      orbital_list=[orb]*len(label_list))
        cohp_plot_static.add_cohp(orb, cohp)
cohp_plot_static.get_plot(ylim=[-15,2]);

In [ ]:
# Using interactive plotter to add relevant cohps
interactive_cohp_plot = InteractiveCohpPlotter()

In [ ]:
interactive_cohp_plot.add_all_relevant_cohps(analyse=analyse, label_resolved=False,orbital_resolved=True,suffix='')

In [ ]:
fig = interactive_cohp_plot.get_plot()
fig.show(renderer='notebook')

Due to sphinx rendering limitations for Plotly figures, the output is not directly visible within the notebook; please use the link to see the [interactive orbital resolved plot](tutorial_assets/CdF2_orb.html)

### Plot DOS from Lobster

In [ ]:
# Load Lobster DOS
directory = Path(".") / ".." / ".." / "tests" / "test_data" / "NaCl_comp_range"
dos = Doscar(doscar=directory / 'DOSCAR.lobster.gz',
             structure_file=directory / 'POSCAR.gz')

```python
# Load Lobster DOS (Change this cell block type to Code when executing locally)
directory = Path("LobsterPy") / "tests" / "test_data" / "NaCl_comp_range"
dos = Doscar(doscar=directory / 'DOSCAR.lobster.gz',
             structure_file=directory / 'POSCAR.gz')
```

Plot total, element and spd dos

In [ ]:
style.use('default') # Complete reset the matplotlib figure style
style.use(get_style_list()[0]) # Use the LobsterPy style sheet for the generated plots

dos_plotter = PlainDosPlotter(summed=True, stack=False, sigma=None)
dos_plotter.add_dos(dos=dos.completedos, label='Total DOS')
dos_plotter.add_dos_dict(dos_dict=dos.completedos.get_element_dos()) # Add element dos
dos_plotter.add_dos_dict(dos_dict=dos.completedos.get_spd_dos()) # add spd dos
dos_plotter.get_plot(xlim=[-10, 3]);

Plotting DOS at particular site and orbital

In [ ]:
dos_plotter = PlainDosPlotter(summed=True, stack=False, sigma=0.03)
dos_plotter.add_site_orbital_dos(dos = dos.completedos, site_index=0, orbital='3s')
dos_plotter.get_plot(xlim=[-10, 3]);

## Generate structure graph objects with LOBSTER data

In [ ]:
from lobsterpy.structuregraph.graph import LobsterGraph

Below code snippet will generate a networkx graph object with ICOHP, ICOOP, and ICOBI data as edge properties and charges as node properties.

In [ ]:
graph_NaCl_all = LobsterGraph(
    path_to_poscar=directory / "POSCAR.gz",
    path_to_charge=directory / "CHARGE.lobster.gz",
    path_to_cohpcar=directory / "COHPCAR.lobster.gz",
    path_to_icohplist=directory / "ICOHPLIST.lobster.gz",
    add_additional_data_sg=True,
    path_to_icooplist=directory / "ICOOPLIST.lobster.gz",
    path_to_icobilist=directory / "ICOBILIST.lobster.gz",
    path_to_madelung=directory / "MadelungEnergies.lobster.gz",
    which_bonds="all",
    start=None,
)

```python
#### (Change this cell block type to Code or copy it from here when executing locally)
graph_NaCl_all = LobsterGraph(
    path_to_poscar=directory / "POSCAR.gz",
    path_to_charge=directory / "CHARGE.lobster.gz",
    path_to_cohpcar=directory / "COHPCAR.lobster.gz",
    path_to_icohplist=directory / "ICOHPLIST.lobster.gz",
    add_additional_data_sg=True,
    path_to_icooplist=directory / "ICOOPLIST.lobster.gz",
    path_to_icobilist=directory / "ICOBILIST.lobster.gz",
    path_to_madelung=directory / "MadelungEnergies.lobster.gz",
    which_bonds="all",
    start=None,
)
```

In [ ]:
graph_NaCl_all.sg.graph.nodes.data() # view node data

In [ ]:
graph_NaCl_all.sg.graph.edges.data() # view edge data

## Featurizer usage examples (Generates features from LOBSTER data for ML studies)

:::{note}
To use the batch featurizers, the path to the parent directory containing LOBSTER calculation outputs needs to be provided. For example, your directory structure needs to be like this:

parent_dir/lobster_calc_output_dir_for_compound_1/
parent_dir/lobster_calc_output_dir_for_compound_2/
parent_dir/lobster_calc_output_dir_for_compound_3/

the `lobster_calc_output_dir_for_compound_*` directory should contain all your LOBSTER outputs and POSCAR file.

In such a case `path_to_lobster_calcs="parent_dir"` needs to be set 
:::

In [ ]:
from lobsterpy.featurize.batch import (BatchCoxxFingerprint, BatchDosFeaturizer,
                                       BatchSummaryFeaturizer, BatchStructureGraphs)

### BatchCoxxFingerprint

`BatchCoxxFingerprint` provides a convenient way to directly generate fingerprint objects from COHP / COBI/ COOPCAR.lobster data. Generating fingerprints specifically for `bonding`, `antibonding`, and `overall` interactions is feasible. 

One can also generate a pair-wise fingerprint similarity matrix dataframe (currently, only simple vector dot product or Tanimoto index are implemented)


In [ ]:
# Initialize the batch COXX featurizer 
fp_cohp_bonding = BatchCoxxFingerprint(
            path_to_lobster_calcs=directory / ".." / "Featurizer_test_data" / "Lobster_calcs",
            e_range=[-15, 0], 
            feature_type="bonding",
            normalize=True, # affects only the fingerprint similarity matrix computation
            tanimoto=True, # affects only the fingerprint similarity matrix computation
            n_jobs=3,
        fingerprint_for='cohp' # changing this to cobi/coop will result in reading cobicar/coopcar file
        )

```python
## Initialize batch COXX featurizer (Change this cell block type to Code and remove formatting when executing locally)
fp_cohp_bonding = BatchCoxxFingerprint(
    path_to_lobster_calcs=directory / ".." / "Featurizer_test_data" / "Lobster_calcs",
    e_range=[-15, 0], 
    feature_type="bonding",
    normalize=True, # affects only the fingerprint similarity matrix computation
    tanimoto=True, # affects only the fingerprint similarity matrix computation
    n_jobs=3,
    fingerprint_for='cohp' # changing this to cobi/coop will result in reading cobicar/coopcar file
)
```

In [ ]:
# Access the fingerprints dataframe
fp_cohp_bonding.fingerprint_df

In [ ]:
# Get the fingerprints similarity matrix
fp_cohp_bonding.get_similarity_matrix_df()

### BatchDosFeaturizer

`BatchDosFeaturizer` provides a convenient way to extract LOBSTER DOS moment features and fingerprints in the form of pandas dataframe from the LOBSTER calculation directory. The extracted features consist of the following:


1. Element and PDOS center, width, skewness, kurtosis, and edges 
2. PDOS or total DOS fingerprint objects

In [ ]:
# Initialize the batch DOS featurizer
batch_dos = BatchDosFeaturizer(path_to_lobster_calcs=directory / ".." / "Featurizer_test_data" / "Lobster_calcs", # path to parent lobster calcs
            use_lso_dos=True, # will enforce using DOSCAR.LSO.lobster
            add_element_dos_moments=True, # set to false to not have element moments dos features 
            e_range=None, # setting this to none results in features computed for the entire energy range 
            fingerprint_type="summed_pdos", # fingerprint type (s,p,d,f, summed_pdos)
            n_bins=256,
            n_jobs=3,)

```python
## Initialize batch DOS featurizer (Change this cell block type to Code and remove formatting when executing locally)
batch_dos = BatchDosFeaturizer(path_to_lobster_calcs=directory / ".." / "Featurizer_test_data" / "Lobster_calcs", # path to parent lobster calcs
            use_lso_dos=True, # will enforce using DOSCAR.LSO.lobster
            add_element_dos_moments=True, # set to false to not have element moments dos features 
            e_range=None, # setting this to none results in features computed for the entire energy range 
            fingerprint_type="summed_pdos", # fingerprint type (s,p,d,f, summed_pdos)
            n_bins=256,
            n_jobs=3,)
```

In [ ]:
# get the DOS moments df
batch_dos.get_df()

In [ ]:
# get the DOS fingerprints df
batch_dos.get_fingerprints_df()

### BatchSummaryFeaturizer

`BatchSummaryFeaturizer` provides a convenient way to extract summary stats as pandas dataframe from the LOBSTER calculation directory. The summary stats consist of the following:

1. ICOHP, bonding, antibonding percent (mean, min, max, standard deviation) of relevant bonds from LobsterPy analysis (Orbital-wise analysis stats data can also be included: Optional)
2. Weighted ICOHP ( ICOOP/ ICOBI: Optional)
3. COHP center, width, skewness, kurtosis, edge (COOP/ COBI: Optional)
4. Ionicity and Madelung energies for the structure based on Mulliken and Loewdin charges

In [ ]:
# Initialize batch summary featurizer
summary_features = BatchSummaryFeaturizer(
            path_to_lobster_calcs=directory / ".." / "Featurizer_test_data" / "Lobster_calcs",
            bonds="all",
            include_cobi_data=False,
            include_coop_data=False,
            e_range=[-15, 0],
            n_jobs=3,
        )

```python
## Initialize batch summary featurizer (Change this cell block type to Code and remove formatting when executing locally)
summary_features = BatchSummaryFeaturizer(
            path_to_lobster_calcs=directory / ".." / "Featurizer_test_data" / "Lobster_calcs",
            bonds="all",
            include_cobi_data=False,
            include_coop_data=False,
            e_range=[-15, 0],
            n_jobs=3,
        )
```

In [ ]:
# get summary stats features 
summary_features.get_df()

### BatchStructureGraphs

`BatchStructureGraphs` provides a convenient way to generate structure graph objects with LOBSTER data in the form of pandas dataframe from a set of the LOBSTER calculation directories.

In [ ]:
batch_sg = BatchStructureGraphs(path_to_lobster_calcs=directory / ".." / "Featurizer_test_data" / "Lobster_calcs",
                                add_additional_data_sg=True,
                                which_bonds='all',
                                n_jobs=3,
                                start=None)

```python
## Initialize batch structure graphs featurizer (Change this cell block type to Code and remove formatting when executing locally)
batch_sg = BatchStructureGraphs(path_to_lobster_calcs=directory / ".." / "Featurizer_test_data" / "Lobster_calcs",
                                add_additional_data_sg=True,
                                which_bonds='all',
                                n_jobs=3,
                                start=None)
```

In [ ]:
# get the structure graphs df
batch_sg.get_df()